In [1]:
from yahoo_finance import Share
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import random

In [2]:
def get_prices(share_symbol, start_date, end_date, cache_filename='stock_price.npy'):
    try:
        stock_prices = np.load(cache_filename)
    except IOError:
        share = Share(share_symbol)
        stock_hist = share.get_historical(start_date, end_date)
        stock_prices = [stock_price['Open'] for stock_price in stock_hist]
        np.save(cache_filename, stock_prices)
        
    return stock_prices

In [3]:
def plot_prices(prices):
    plt.title('Opening stock prices')
    plt.xlabel('day')
    plt.ylabel('price ($)')
    plt.plot(prices)
    plt.savefig('prices.png')

In [8]:
ms = Share('MSFT')
print(ms.get_trade_datetime())
prices = get_prices('MSFT', '1992-07-22', '2017-05-04')
plot_prices(prices)

2017-05-04 16:56:00 UTC+0000


In [18]:
class DecisionPolicy:
    def select_action(self, current_state, step):
        pass
    
    def update_q(self, state, action, reward, next_state):
        pass
    

In [19]:
class RandomDecisionPolicy(DecisionPolicy):
    def __init__(self, actions):
        self.actions = actions
    
    def select_action(self, current_state, step):
        action = self.actions[random.randint(0, len(self.actions) - 1)]
        return action

In [16]:
def run_simulation(policy, initial_budget, initial_num_stocks, prices, hist, debug=False):
    budget = initial_budget
    num_stocks = initial_num_stocks
    share_value = 0
    transitions = list()
    for i in range(len(prices) - hist - 1):
        if i % 100 == 0:
            print('progress {:.2f}%'.format(float(100*i) / (len(prices) - hist - 1)))
        current_state = np.asmatrix(np.hstack((prices[i:i+hist], budget, num_stocks)))
        current_portfolio = budget + num_stocks * share_value
        action = policy.select_action(current_state, i)
        share_value = float(prices[i + hist +1])
        if action == 'Buy' and budget >= share_value:
            budget -= share_value
            num_stocks += 1
        elif action == 'Sell' and num_stocks > 0:
            budget += share_value
            num_stocks -= 1
        else:
            action = 'Hold'
        new_portfolio = budget + num_stocks * share_value
        reward = new_portfolio - current_portfolio
        next_state = np.asmatrix(np.hstack((prices[i+1:i+hist+1], budget, num_stocks)))
        transitions.append((current_state, action, reward, next_state))
        policy.update_q(current_state, action, reward, next_state)
    
    portfolio = budget + num_stocks * share_value
    if debug:
        print('${}\t{} shares'.format(budget, num_stocks))
    return portfolio

In [12]:
def run_simulations(policy, budget, num_stocks, prices, hist):
    num_tries = 10
    final_portfolios = list()
    for i in range(num_tries):
        final_portfolio = run_simulation(policy, budget, num_stocks, prices, hist)
        final_portfolios.append(final_portfolio)
    avg, std = np.mean(final_portfolios), np.std(final_portfolios)
    return avg, std

In [20]:
actions = ['Buy', 'Sell', 'Hold']
hist = 200
policy = RandomDecisionPolicy(actions)
budget = 1000.0
num_stocks = 0
avg, std = run_simulations(policy, budget, num_stocks, prices, hist)
print(avg, std)

progress 0.00%
progress 1.65%
progress 3.31%
progress 4.96%
progress 6.62%
progress 8.27%
progress 9.93%
progress 11.58%
progress 13.24%
progress 14.89%
progress 16.55%
progress 18.20%
progress 19.86%
progress 21.51%
progress 23.17%
progress 24.82%
progress 26.48%
progress 28.13%
progress 29.79%
progress 31.44%
progress 33.10%
progress 34.75%
progress 36.41%
progress 38.06%
progress 39.72%
progress 41.37%
progress 43.02%
progress 44.68%
progress 46.33%
progress 47.99%
progress 49.64%
progress 51.30%
progress 52.95%
progress 54.61%
progress 56.26%
progress 57.92%
progress 59.57%
progress 61.23%
progress 62.88%
progress 64.54%
progress 66.19%
progress 67.85%
progress 69.50%
progress 71.16%
progress 72.81%
progress 74.47%
progress 76.12%
progress 77.78%
progress 79.43%
progress 81.09%
progress 82.74%
progress 84.40%
progress 86.05%
progress 87.70%
progress 89.36%
progress 91.01%
progress 92.67%
progress 94.32%
progress 95.98%
progress 97.63%
progress 99.29%
progress 0.00%
progress 1.65%
p

In [27]:
class QLearningDecisionPolicy(DecisionPolicy):
    def __init__(self, actions, input_dim):
        self.epsilon = 0.9
        self.gamma = 0.001
        self.actions = actions
        output_dim = len(actions)
        h1_dim = 200

        self.x = tf.placeholder(tf.float32, [None, input_dim])
        self.y = tf.placeholder(tf.float32, [output_dim])
        W1 = tf.Variable(tf.random_normal([input_dim, h1_dim]))
        b1 = tf.Variable(tf.constant(0.1, shape=[h1_dim]))
        h1 = tf.nn.relu(tf.matmul(self.x, W1) + b1)
        W2 = tf.Variable(tf.random_normal([h1_dim, output_dim]))
        b2 = tf.Variable(tf.constant(0.1, shape=[output_dim]))
        self.q = tf.nn.relu(tf.matmul(h1, W2) + b2)

        loss = tf.square(self.y - self.q)
        self.train_op = tf.train.AdagradOptimizer(0.01).minimize(loss)
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def select_action(self, current_state, step):
        threshold = min(self.epsilon, step / 1000.)
        if random.random() < threshold:
            action_q_vals = self.sess.run(self.q, feed_dict={self.x: current_state})
            action_idx = np.argmax(action_q_vals)
            action = self.actions[action_idx]
        else:
            action = self.actions[random.randint(0, len(self.actions) - 1)]
        return action

    def update_q(self, state, action, reward, next_state):
        action_q_vals = self.sess.run(self.q, feed_dict={self.x: state})
        next_action_q_vals = self.sess.run(self.q, feed_dict={self.x: next_state})
        next_action_idx = np.argmax(next_action_q_vals)
        action_q_vals[0, next_action_idx] = reward + self.gamma * next_action_q_vals[0, next_action_idx]
        action_q_vals = np.squeeze(np.asarray(action_q_vals))
        self.sess.run(self.train_op, feed_dict={self.x: state, self.y: action_q_vals})

In [28]:
hist = 200
policy = QLearningDecisionPolicy(actions, hist + 2)
budget = 1000.0
num_stocks = 0
avg, std = run_simulations(policy, budget, num_stocks, prices, hist)
print(avg, std)

progress 0.00%
progress 1.65%
progress 3.31%
progress 4.96%
progress 6.62%
progress 8.27%
progress 9.93%
progress 11.58%
progress 13.24%
progress 14.89%
progress 16.55%
progress 18.20%
progress 19.86%
progress 21.51%
progress 23.17%
progress 24.82%
progress 26.48%
progress 28.13%
progress 29.79%
progress 31.44%
progress 33.10%
progress 34.75%
progress 36.41%
progress 38.06%
progress 39.72%
progress 41.37%
progress 43.02%
progress 44.68%
progress 46.33%
progress 47.99%
progress 49.64%
progress 51.30%
progress 52.95%
progress 54.61%
progress 56.26%
progress 57.92%
progress 59.57%
progress 61.23%
progress 62.88%
progress 64.54%
progress 66.19%
progress 67.85%
progress 69.50%
progress 71.16%
progress 72.81%
progress 74.47%
progress 76.12%
progress 77.78%
progress 79.43%
progress 81.09%
progress 82.74%
progress 84.40%
progress 86.05%
progress 87.70%
progress 89.36%
progress 91.01%
progress 92.67%
progress 94.32%
progress 95.98%
progress 97.63%
progress 99.29%
progress 0.00%
progress 1.65%
p